In [1]:
import getpass
import pymysql
import unidecode
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

In [3]:
password=getpass.getpass()

········


In [4]:
tipo_carta=['Diego_Restrepo', 'cartas_gfif']
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[0])

In [5]:
df=pd.read_sql('select * from cartas;', con=conn)

In [6]:
sg=pd.read_sql('select * from cartas_sign;', con=conn)

Last entry

## Search in body

In [7]:
df[df.body.str.lower().map(unidecode.unidecode).str.contains('impacto social')]

,id,description,preamble,para,address,signature,opening,body,closing,fecha,link
274,280,,\usepackage{udea}\usepackage[spanish]{babel},Señores COMITÉ TÉCNICO\\\r\nCentro de Investigación de Ciencias Naturales y Exactas,,Diego IP,Respetados señores:,"Solicitamos aval para presentar solicitud al CODI de hacer un aplazamiento a la firma del acta de inicio del proyecto intitulado ``IMPACTO SOCIAL, CIENTÍFICO Y ACTIVIDADES DE VINCULACIÓN CON EL ENTORNO DE CENTROS DE INVESTIGACIÓN DE LA UNIVERSIDAD DE ANTIOQUIA"". Este proyecto fue aprobado por el CODI en el marco de la convocatoria programática del área de ciencias sociales, humanidades y artes 2016 en acta 748 del 2017.\r\n \r\nDebido a que la decisión final de aceptación fue posterior a la ...",Agradeciendo su colaboración en este sentido,2017-09-06,http://gfif.udea.edu.co/cartas/cartas.php?step=1&id=


In [8]:
c=df.loc[df.index.values[-1]]

In [9]:
c

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             280
description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [10]:
c.keys().values

array(['id', 'description', 'preamble', 'para', 'address', 'signature',
       'opening', 'body', 'closing', 'fecha', 'link'], dtype=object)

In [11]:
for k in c.keys():
    print(k)
    if c[k]==None:
        c[k]=''

id
description
preamble
para
address
signature
opening
body
closing
fecha
link


In [12]:
latexfile=r'''
%'''+c.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+c.preamble+r'''
'''+c.address+r'''
\signature{'''+sg[sg.signature==c.signature].sign.values[0]+r'''}
\begin{document}
\begin{letter}{'''+c.para+r'''}

\opening{'''+c.opening+r'''} 

'''+c.body+r'''

\closing{'''+c.closing+r'''}
\end{letter}
\end{document}
'''

In [13]:
import tempfile
import subprocess
import sys

In [14]:
f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)

In [15]:
print(latexfile)


%
\documentclass{letter}
\usepackage[utf8]{inputenc}
\usepackage{udea}\usepackage[spanish]{babel}

\signature{Diego Restrepo\\
Investigador Principal\\
Profesor  de Tiempo Completo\\
Instituto de F\'\i sica\\
Universidad de Antioquia}
\begin{document}
\begin{letter}{Señores COMITÉ TÉCNICO\\
Centro de Investigación de Ciencias Naturales y Exactas}

\opening{Respetados señores:} 

Solicitamos aval para presentar solicitud al CODI de hacer un aplazamiento a la firma del acta de inicio del proyecto intitulado ``IMPACTO SOCIAL, CIENTÍFICO Y ACTIVIDADES DE VINCULACIÓN CON EL ENTORNO DE CENTROS DE INVESTIGACIÓN DE LA UNIVERSIDAD DE ANTIOQUIA". Este proyecto fue aprobado por el CODI en el marco de la convocatoria programática del área de ciencias sociales, humanidades y artes 2016 en acta 748 del 2017.
 
Debido a que la decisión final de aceptación fue posterior a la negociación de los planes de trabajo de algunos investigadores que hacen parte, debemos postergar el inicio hasta la firma de u

In [16]:
f.write(latexfile)

1843

In [17]:
f.name

'/tmp/tmpbi758bh4.tex'

In [18]:
f.close()

In [19]:
lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

In [23]:
pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
    pdf='carta_{:d}.pdf'.format(c['id'])
    dwld_dir='downloads'
    kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
    print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
    for suffix in  ['.aux','.log','.out','.tex']:
        subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
else:
    sys.exit('LaTeX ERROR:')    

output in http://gfif.udea.edu.co/downloads/carta_280.pdf
